## TP2 - Exercice 2 - 15/10/2025

### Objectif :

- Étudier et appliquer des méthodes d’apprentissage supervisé pour prévoir des valeurs ou classer des données :
- Comparer les performances du KNN selon k et le type de distance. 
- Développer une régression linéaire pour prédire le prix des appartements via normalisation et descente de gradient.
- Identifier le meilleur modèle et analyser l’influence des caractéristiques.


1. Donner la bibliothèque et les fonctions nécessaires sur python qui permet
d’appliquer l’algorithme KNN. 


In [1]:
# Bibliothèque principale
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

# Fonctions pour préparer et évaluer les données
from sklearn.preprocessing import StandardScaler   # Pour normaliser les données
from sklearn.model_selection import train_test_split  # Pour séparer données train/test
from sklearn.metrics import accuracy_score, mean_squared_error  # Pour mesurer la performance


- KNeighborsClassifier → pour classification (prévoir une catégorie comme résultat)
- KNeighborsRegressor → pour régression (prévoir une valeur continue comme résultat)

- StandardScaler : met les données sur une échelle comparable (très important pour KNN).
- train_test_split : divise tes données en ensemble d’entraînement et test.
- accuracy_score : mesure la précision pour la classification.
- mean_squared_error : mesure l’erreur pour la régression.

2. Donner les fonctions qui permettent de calculer les différents types des distances. 


In [4]:
import numpy as np

def euclidean_distance(a, b):
    """
    Distance Euclidienne entre deux vecteurs a et b
    """
    return np.linalg.norm(a - b)


def manhattan_distance(a, b):
    """
    Distance de Manhattan entre deux vecteurs a et b
    """
    return np.sum(np.abs(a - b))


3. Donner quelques bases de données utilisées pour exécuter des benchmark. 


1️⃣ Forest CoverType (Classification multiclasse)

- Description : Prédit le type de couverture forestière à partir de caractéristiques topographiques et environnementales (altitude, pente, aspect, type de sol, etc.).

In [14]:
from sklearn.datasets import fetch_covtype

# Chargement du dataset
forest = fetch_covtype()
X_forest, y_forest = forest.data, forest.target

# Aperçu
print("Nombre d'échantillons :", X_forest.shape[0])
print("Nombre de caractéristiques :", X_forest.shape[1])
print("Classes :", set(y_forest))


Nombre d'échantillons : 581012
Nombre de caractéristiques : 54
Classes : {1, 2, 3, 4, 5, 6, 7}


2️⃣ Diabetes (Régression)

- Description : Mesure de progression de la maladie du diabète selon différentes caractéristiques.
- Variable cible : Indice de progression

In [6]:
from sklearn.datasets import load_diabetes

# Chargement du dataset
diabetes = load_diabetes()
X_diabetes, y_diabetes = diabetes.data, diabetes.target

# Aperçu
print("Nombre d'échantillons :", X_diabetes.shape[0])
print("Nombre de caractéristiques :", X_diabetes.shape[1])
print("Valeurs cibles (exemple) :", y_diabetes[:5])


Nombre d'échantillons : 442
Nombre de caractéristiques : 10
Valeurs cibles (exemple) : [151.  75. 141. 206. 135.]


4. Utilisant plusieurs types de distances dresser un tableau des résultats selon les deux
hyper paramètres considérés dans l’exercice précèdent. 

In [15]:
import pandas as pd

# -------------------------------
# Préparation des datasets
# -------------------------------

# Forest CoverType - classification
forest = fetch_covtype()
X_forest, y_forest = forest.data, forest.target

# Sous-échantillonnage pour test rapide (ex : 5000 échantillons)
np.random.seed(42)
indices = np.random.choice(X_forest.shape[0], 5000, replace=False)
X_forest_sample = X_forest[indices]
y_forest_sample = y_forest[indices]

# Split train/test
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_forest_sample, y_forest_sample, test_size=0.3, random_state=42
)

# Normalisation
scaler_f = StandardScaler()
X_train_f = scaler_f.fit_transform(X_train_f)
X_test_f = scaler_f.transform(X_test_f)

# Diabetes - régression
diabetes = load_diabetes()
X_diabetes, y_diabetes = diabetes.data, diabetes.target
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_diabetes, y_diabetes, test_size=0.3, random_state=42
)

# Normalisation Diabetes
scaler_d = StandardScaler()
X_train_d = scaler_d.fit_transform(X_train_d)
X_test_d = scaler_d.transform(X_test_d)

# -------------------------------
# Définition des hyper-paramètres
# -------------------------------
k_values = [1, 3]                  
distances = ['euclidean', 'manhattan']  

# -------------------------------
# KNN Classification - Forest
# -------------------------------
results_forest = []

for dist in distances:
    for k in k_values:
        clf = KNeighborsClassifier(n_neighbors=k, metric=dist)
        clf.fit(X_train_f, y_train_f)
        y_pred_f = clf.predict(X_test_f)
        acc = accuracy_score(y_test_f, y_pred_f)
        results_forest.append((k, dist, acc))

df_forest = pd.DataFrame(results_forest, columns=['k', 'Distance', 'Accuracy'])
df_forest = df_forest.pivot(index='k', columns='Distance', values='Accuracy')
print("===== Forest CoverType Dataset (Classification) =====")
print(df_forest)

# -------------------------------
# KNN Régression - Diabetes
# -------------------------------
results_diabetes = []

for dist in distances:
    for k in k_values:
        reg = KNeighborsRegressor(n_neighbors=k, metric=dist)
        reg.fit(X_train_d, y_train_d)
        y_pred_d = reg.predict(X_test_d)
        mse = mean_squared_error(y_test_d, y_pred_d)
        results_diabetes.append((k, dist, mse))

df_diabetes = pd.DataFrame(results_diabetes, columns=['k', 'Distance', 'MSE'])
df_diabetes = df_diabetes.pivot(index='k', columns='Distance', values='MSE')
print("\n===== Diabetes Dataset (Régression) =====")
print(df_diabetes)

===== Forest CoverType Dataset (Classification) =====
Distance  euclidean  manhattan
k                             
1          0.694000   0.694000
3          0.692667   0.695333

===== Diabetes Dataset (Régression) =====
Distance    euclidean    manhattan
k                                 
1         5966.646617  6320.225564
3         3630.467001  3428.751044


- Forest CoverType (Classification) : La précision est d’environ 69 % pour toutes les combinaisons de k et de distance, montrant que KNN est relativement stable sur ce dataset.

- Diabetes (Régression) : L’erreur MSE diminue en passant de k=1 à k=3, et la distance Manhattan donne légèrement de meilleurs résultats, indiquant que augmenter k réduit le bruit et l’impact des points atypiques.

5. Développer un modèle de régression linéaire complet pour la classification des
appartements utilisant la normalisation par les valeurs suivantes : 500, 10, 10 et 1000
et un pas d’apprentissage 0,08.

In [22]:
  # -------------------------------
# 1️⃣ Données des appartements
# -------------------------------
data = {
    'Surface': [135, 105, 125, 90, 105, 125, 85, 140],
    'Etage': [1, 2, 2, 1, 3, 0, 3, 0],
    'Nb_pieces': [4, 3, 4, 2, 4, 4, 2, 4],
    'Prix': [240, 165, 210, 150, 190, 240, 140, 250]
}

df = pd.DataFrame(data)

print("=== DataFrame original ===")
print(df)

# -------------------------------
# 2️⃣ Normalisation des caractéristiques
# -------------------------------
X = df[['Surface', 'Etage', 'Nb_pieces']].values
y = df['Prix'].values.reshape(-1, 1)

norm_values = np.array([500, 10, 10])
X_norm = X / norm_values   # normalisation des caractéristiques
y_norm = y / 1000          # normalisation du prix

# Création d'un DataFrame pour visualiser les valeurs normalisées
df_norm = pd.DataFrame(X_norm, columns=['Surface_norm', 'Etage_norm', 'Nb_pieces_norm'])
df_norm['Prix_norm'] = y_norm

print("\n=== DataFrame normalisé ===")
print(df_norm)

# -------------------------------
# 3️⃣ Ajouter le biais pour le modèle linéaire
# -------------------------------
m = X_norm.shape[0]  # nombre d'exemples
X_bias = np.hstack([np.ones((m, 1)), X_norm])  # ajout de la colonne 1 pour theta0

n = X_bias.shape[1]  # nombre de paramètres
theta = np.zeros((n, 1))  # initialisation des paramètres à 0

print("\n=== Matrice X avec biais ===")
print(X_bias)
print("\n=== Paramètres initiaux theta ===")
print(theta)

# -------------------------------
# 4️⃣ Définir la fonction de coût (Mean Squared Error)
# -------------------------------
def compute_cost(X, y, theta):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1/(2*m)) * np.sum((predictions - y)**2)
    return cost

# Vérification du coût initial
initial_cost = compute_cost(X_bias, y_norm, theta)
print("\nCoût initial :", initial_cost)

# -------------------------------
# -------------------------------
# 5️⃣ Descente de gradient avec affichage partiel
# -------------------------------
alpha = 0.08      # pas d’apprentissage
iterations = 1000 # nombre d’itérations
cost_history = []

for i in range(iterations):
    predictions = X_bias.dot(theta)
    gradient = (1/m) * X_bias.T.dot(predictions - y_norm)
    theta = theta - alpha * gradient
    cost = compute_cost(X_bias, y_norm, theta)
    cost_history.append(cost)
    
    # Affichage tous les 100 itérations
    if (i+1) % 100 == 0:
        print(f"Iteration {i+1:4d} | Coût = {cost:.8f} | Theta = {theta.ravel()}")

print("\n=== Paramètres finaux theta ===")
print(theta)
print("\nCoût final :", cost_history[-1])

# -------------------------------
# 6️⃣ Prédiction avec le modèle
# -------------------------------
y_pred_norm = X_bias.dot(theta)
y_pred = y_pred_norm * 1000  # remise à l’échelle du prix

df['Prix_Pred'] = y_pred
print("\n=== Prix réels vs prédits ===")
print(df)

=== DataFrame original ===
   Surface  Etage  Nb_pieces  Prix
0      135      1          4   240
1      105      2          3   165
2      125      2          4   210
3       90      1          2   150
4      105      3          4   190
5      125      0          4   240
6       85      3          2   140
7      140      0          4   250

=== DataFrame normalisé ===
   Surface_norm  Etage_norm  Nb_pieces_norm  Prix_norm
0          0.27         0.1             0.4      0.240
1          0.21         0.2             0.3      0.165
2          0.25         0.2             0.4      0.210
3          0.18         0.1             0.2      0.150
4          0.21         0.3             0.4      0.190
5          0.25         0.0             0.4      0.240
6          0.17         0.3             0.2      0.140
7          0.28         0.0             0.4      0.250

=== Matrice X avec biais ===
[[1.   0.27 0.1  0.4 ]
 [1.   0.21 0.2  0.3 ]
 [1.   0.25 0.2  0.4 ]
 [1.   0.18 0.1  0.2 ]
 [1.   0.21 

**Observation :**

- Les prix prédits sont proches des prix réels, même avec seulement 8 appartements.
- La descente de gradient a convergé rapidement, le coût final est très faible (~0,00012), indiquant un bon ajustement du modèle.
- Les coefficients finaux (θ) montrent l’influence de chaque variable :
- Surface : plus grande influence positive sur le prix
- Étages : influence légèrement négative ou faible
- Nombre de pièces : influence positive sur le prix
- Le modèle linéaire peut donc approximer correctement les prix pour ce petit dataset après normalisation.

6. Donner le tableau des performances du modèle pour différentes valeurs des hyper
parametres. 


In [19]:

# Données déjà normalisées
X_bias = np.hstack([np.ones((X_norm.shape[0],1)), X_norm])
y_norm = y / 1000
m = X_bias.shape[0]

# Hyperparamètres à tester
alphas = [0.01, 0.05, 0.08, 0.1]
iterations_list = [500, 1000, 2000]

# Tableau pour stocker les résultats
results = []

for alpha in alphas:
    for iterations in iterations_list:
        theta = np.zeros((X_bias.shape[1],1))
        # Descente de gradient
        for i in range(iterations):
            gradient = (1/m) * X_bias.T.dot(X_bias.dot(theta) - y_norm)
            theta = theta - alpha * gradient
        # Calcul du coût final
        cost_final = (1/(2*m)) * np.sum((X_bias.dot(theta) - y_norm)**2)
        results.append((alpha, iterations, cost_final))

# Création DataFrame
df_perf = pd.DataFrame(results, columns=['Alpha', 'Iterations', 'MSE'])
print("=== Tableau des performances ===")
print(df_perf.pivot(index='Alpha', columns='Iterations', values='MSE'))

=== Tableau des performances ===
Iterations      500       1000      2000
Alpha                                   
0.01        0.000603  0.000530  0.000412
0.05        0.000365  0.000207  0.000088
0.08        0.000257  0.000118  0.000050
0.10        0.000207  0.000088  0.000042


7. Donner l’expression du meilleur modèle. 

In [20]:
# Descente de gradient
# -------------------------------
alpha = 0.10       # meilleur alpha
iterations = 2000  # meilleur nombre d'itérations
theta = np.zeros((X_bias.shape[1],1))

for i in range(iterations):
    gradient = (1/m) * X_bias.T.dot(X_bias.dot(theta) - y_norm)
    theta = theta - alpha * gradient

# -------------------------------
# Affichage du θ final
# -------------------------------
theta_final = theta.flatten()
param_names = ['θ0 (intercept)', 'θ1 (Surface)', 'θ2 (Etage)', 'θ3 (Nb_pieces)']

df_theta = pd.DataFrame({'Paramètre': param_names, 'Valeur': theta_final})
print("=== θ final du meilleur modèle ===")
print(df_theta)

=== θ final du meilleur modèle ===
        Paramètre    Valeur
0  θ0 (intercept)  0.106904
1    θ1 (Surface)  0.130104
2      θ2 (Etage) -0.154647
3  θ3 (Nb_pieces)  0.251722


##### 7️⃣ Meilleur modèle de régression linéaire

Le meilleur modèle correspond à **α = 0.10** et **2000 itérations**, car il donne le **MSE le plus faible** sur notre dataset.

##### 1️⃣ Paramètres finaux (normalisés)
| Paramètre | Valeur |
|-----------|--------|
| θ0 (intercept) | 0.1069 |
| θ1 (Surface)   | 0.1301 |
| θ2 (Etage)     | -0.1546 |
| θ3 (Nb_pieces) | 0.2517 |

##### 2️⃣ Équation du modèle (normalisée)

$$
\hat{y}_{norm} = 0.1069 + 0.1301 \cdot X_1^{norm} - 0.1546 \cdot X_2^{norm} + 0.2517 \cdot X_3^{norm}
$$

avec :  

$$
X_1^{norm} = \frac{\text{Surface}}{500}, \quad 
X_2^{norm} = \frac{\text{Etage}}{10}, \quad 
X_3^{norm} = \frac{\text{Nb\_pieces}}{10}
$$  

##### 3️⃣ Équation du modèle en prix réel

$$
\hat{y} = 1000 \cdot \Big(0.1069 + 0.1301 \frac{\text{Surface}}{500} - 0.1546 \frac{\text{Etage}}{10} + 0.2517 \frac{\text{Nb\_pieces}}{10}\Big)
$$

**Observation :**  
- La **Surface** et le **Nombre de pièces** augmentent significativement le prix,  
- L’**Étage** a un effet légèrement négatif sur le prix,  
- Ce modèle minimise l’erreur sur notre dataset après normalisation et descente de gradient, donnant ainsi les meilleures prédictions pour ces appartements.
